In [34]:
import json

data_list = []
for filename in json_files:
    with open(filename) as f:
        data_list.append(json.load(f))

In [5]:
products = [
    {
  "product_id": "12345",
  "product_name": "Widget A",
  "price": 19.99,
  "description": "A high-quality widget.",
  "manufacturer": "Company X",
  "stock_quantity": None
},

{
  "SKU": "SKU-54321",
  "product_name": "SuperWidget",
  "price": 24.95,
  "description": "The ultimate super widget.",
  "manufacturer": "Brand Y",
  "inventory": 150
},

{
  "product_id": "12345",
  "product_name": "Widget A",
  "price": 19.99,
  "description": "A high-quality widget.",
  "manufacturer": "Company X",
  "stock_quantity": 75
}

]

# Save each product as a separate JSON file
for i, product in enumerate(products):
    filename = f"product_{i+1}.json"
    with open(filename, 'w') as outfile:
        json.dump(product, outfile)

print("JSON data saved to individual files successfully!")

JSON data saved to individual files successfully!


In [14]:
def standardize_data(product_data):
    # Ensure consistent keys and handle missing values
    standardized_data = {
        'product_id': product_data.get('product_id', product_data.get('SKU', '')),
        'product_name': product_data.get('product_name', 'Unknown Product'),
        'price': product_data.get('price', 0),
        'description': product_data.get('description', 'No description available'),
        'manufacturer': product_data.get('manufacturer', 'Unknown Manufacturer'),
        'stock_quantity': product_data.get('stock_quantity', product_data.get('inventory', 0))
    }
    return standardized_data

In [37]:
standardized_data = [standardize_data(item) for item in data_list]
df = pd.DataFrame(standardized_data)
df

,product_id,product_name,price,description,manufacturer,stock_quantity
0,12345,Widget A,19.99,A high-quality widget.,Company X,NaN
1,SKU-54321,SuperWidget,24.95,The ultimate super widget.,Brand Y,150.0
2,12345,Widget A,19.99,A high-quality widget.,Company X,75.0


In [32]:
import os
directory = '.'  # The directory where the JSON files are stored
all_files = os.listdir(directory)

# Filter the files to include only JSON files with a standardized name pattern
json_files = [file for file in all_files if file.startswith('product_') and file.endswith('.json')]

# Print the list of JSON files
print(json_files)

['product_1.json', 'product_2.json', 'product_3.json']


In [15]:
json_files = ['product_1.json', 'product_2.json', 'product_3.json']
consolidated_data = merge_data(json_files)
#cleaned_data = detect_duplicates(consolidated_data)


TypeError: list indices must be integers or slices, not str

In [25]:
standardized_data = [standardize_data(item) for item in data_list]

# Create a pandas DataFrame from the standardized data
df = pd.DataFrame(standardized_data)


In [38]:
aggregated_df = df.groupby('product_id').agg({
    'product_name': 'first',  # Take the first value for product_name
    'price': 'mean',  # Calculate the mean price
    'description': 'first',  # Take the first value for description
    'manufacturer': 'first',  # Take the first value for manufacturer
    'stock_quantity': 'sum'  # Sum the stock_quantity values
}).reset_index()

# Reset the index to make 'product_id' a column again
aggregated_df.reset_index(drop=True, inplace=True)

aggregated_df

,product_id,product_name,price,description,manufacturer,stock_quantity
0,12345,Widget A,19.99,A high-quality widget.,Company X,75.0
1,SKU-54321,SuperWidget,24.95,The ultimate super widget.,Brand Y,150.0


In [29]:
aggregated_df

,product_id,product_name,price,description,manufacturer,stock_quantity
0,12345,Widget A,19.99,A high-quality widget.,Company X,75.0
1,SKU-54321,SuperWidget,24.95,The ultimate super widget.,Brand Y,150.0


In [22]:
import pandas as pd

def standardize_data(product_data):
    # Ensure consistent keys and handle missing values
    standardized_data = {
        'product_id': product_data.get('product_id', product_data.get('SKU', '')),
        'product_name': product_data.get('product_name', 'Unknown Product'),
        'price': product_data.get('price', 0),
        'description': product_data.get('description', 'No description available'),
        'manufacturer': product_data.get('manufacturer', 'Unknown Manufacturer'),
        'stock_quantity': product_data.get('stock_quantity', product_data.get('inventory', 0))
    }
    return standardized_data

# Load data from JSON files (assuming they are named "file1.json", "file2.json", and "file3.json")
data_list = []
for filename in ['product_1.json', 'product_2.json', 'product_3.json']:
    with open(filename) as f:
        data_list.append(json.load(f))

In [24]:
df

,product_id,product_name,price,description,manufacturer,stock_quantity,SKU,inventory
0,12345,Widget A,19.99,A high-quality widget.,Company X,NaN,NaN,NaN
1,NaN,SuperWidget,24.95,The ultimate super widget.,Brand Y,NaN,SKU-54321,150.0
2,12345,Widget A,19.99,A high-quality widget.,Company X,75.0,NaN,NaN


In [49]:
consolidated_df = df.groupby('product_id').agg({
    'product_name': 'first', 
    'price': 'first',
    'description': 'first',
    'manufacturer': 'first',
    'stock_quantity': 'sum'  # Sum the stock_quantity values
}).reset_index()

# Store the consolidated data in a central data repository or database.
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("ConsolidatedDataStorage") \
    .getOrCreate()


In [53]:
spark

In [52]:
spark_df = spark.createDataFrame(consolidated_df)

ModuleNotFoundError: No module named 'distutils'

In [41]:
spark_df.write.format("jdbc").option("url", "jdbc:mysql://localhost:3306/testde").option("dbtable", "products").option("user", "root").option("password", "1234qwer").mode("overwrite").save()

AttributeError: 'DataFrame' object has no attribute 'write'